In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Longevity"
cohort = "GSE48264"

# Input paths
in_trait_dir = "../../input/GEO/Longevity"
in_cohort_dir = "../../input/GEO/Longevity/GSE48264"

# Output paths
out_data_file = "../../output/preprocess/Longevity/GSE48264.csv"
out_gene_data_file = "../../output/preprocess/Longevity/gene_data/GSE48264.csv"
out_clinical_data_file = "../../output/preprocess/Longevity/clinical_data/GSE48264.csv"
json_path = "../../output/preprocess/Longevity/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Uppsala Longitudinal Study of Adult Men (ULSAM)"
!Series_summary	"The Uppsala Longitudinal Study of Adult Men is a population-based study aimed at identifying risk factors for cardiovascular disease. At the time of biopsy all subjects were ~ 70yr of age"
!Series_overall_design	"We extracted RNA from muscle tissue taken from 129 subjects, when they were aged ~70yr and characterised as disease-free (note the above average longevity in Swedes born circa 1920 compared with US and UK populations). From these samples, 108 yielded RNA of sufficient quality to profile on Affymetrix gene-chips."
!Series_overall_design	"Only survival data are used in the paper."
!Series_overall_design	"There are no data from cardiovascular disease subjects; we only profiled the healthy subjects and followed for 20yrs."
Sample Characteristics Dictionary:
{0: ['disease-free: disease-free'], 1: ['age(approx): 70 yr'], 2: ['tissue: skeletal muscle biopsy (baseline)'], 3: ['surv

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Background info mentions "Affymetrix gene-chips" which indicates this is gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Longevity):
# The background information indicates this is a longevity study where subjects were followed for 20 years
# From sample characteristics, key 3 shows 'survival' status which can represent longevity
trait_row = 3

# For age:
# All subjects are approximately 70 years old (constant value as per background info)
age_row = None  # Since all subjects have the same age (70), it's not useful for our association study

# For gender:
# No information about gender is provided in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert survival status to binary classification for longevity."""
    if value is None or "None" in value:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # In this study, 'Death' likely indicates lower longevity while 'Hosp' or 'None' 
    # (still in hospital/alive) indicates higher longevity
    if value.lower() == "death":
        return 0  # Death = lower longevity
    elif value.lower() in ["hosp", "none"]:
        return 1  # Hospitalized or None (alive) = higher longevity
    else:
        return None

def convert_age(value):
    """Convert age to continuous value."""
    # Not used in this case as age is constant
    if value is None:
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        # Extract numeric part from strings like "70 yr"
        age_value = float(re.search(r'(\d+)', value).group(1))
        return age_value
    except (ValueError, AttributeError):
        return None

def convert_gender(value):
    """Convert gender to binary (0=female, 1=male)."""
    # Not used in this case as gender information is not available
    if value is None:
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.lower() in ["female", "f"]:
        return 0
    elif value.lower() in ["male", "m"]:
        return 1
    else:
        return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # We've already determined trait data is available
    clinical_selected = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical data
    preview = preview_df(clinical_selected)
    print("Clinical Data Preview:")
    for col, values in preview.items():
        print(f"{col}: {values}")
    
    # Save the clinical data
    clinical_selected.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
GSM1173505: [nan]
GSM1173506: [1.0]
GSM1173507: [0.0]
GSM1173508: [1.0]
GSM1173509: [1.0]
GSM1173510: [0.0]
GSM1173511: [nan]
GSM1173512: [0.0]
GSM1173513: [1.0]
GSM1173514: [1.0]
GSM1173515: [1.0]
GSM1173516: [0.0]
GSM1173517: [1.0]
GSM1173518: [nan]
GSM1173519: [0.0]
GSM1173520: [1.0]
GSM1173521: [nan]
GSM1173522: [1.0]
GSM1173523: [nan]
GSM1173524: [0.0]
GSM1173525: [1.0]
GSM1173526: [1.0]
GSM1173527: [1.0]
GSM1173528: [1.0]
GSM1173529: [nan]
GSM1173530: [1.0]
GSM1173531: [1.0]
GSM1173532: [nan]
GSM1173533: [1.0]
GSM1173534: [1.0]
GSM1173535: [0.0]
GSM1173536: [1.0]
GSM1173537: [nan]
GSM1173538: [1.0]
GSM1173539: [1.0]
GSM1173540: [1.0]
GSM1173541: [nan]
GSM1173542: [0.0]
GSM1173543: [1.0]
GSM1173544: [1.0]
GSM1173545: [1.0]
GSM1173546: [nan]
GSM1173547: [1.0]
GSM1173548: [nan]
GSM1173549: [0.0]
GSM1173550: [nan]
GSM1173551: [nan]
GSM1173552: [1.0]
GSM1173553: [1.0]
GSM1173554: [1.0]
GSM1173555: [1.0]
GSM1173556: [1.0]
GSM1173557: [1.0]
GSM1173558: [1.0]
GSM11

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Uppsala Longitudinal Study of Adult Men (ULSAM)"
Line 1: !Series_geo_accession	"GSE48264"
Line 2: !Series_status	"Public on Aug 05 2015"
Line 3: !Series_submission_date	"Jun 25 2013"
Line 4: !Series_last_update_date	"May 30 2024"
Line 5: !Series_pubmed_id	"26343147"
Line 6: !Series_summary	"The Uppsala Longitudinal Study of Adult Men is a population-based study aimed at identifying risk factors for cardiovascular disease. At the time of biopsy all subjects were ~ 70yr of age"
Line 7: !Series_overall_design	"We extracted RNA from muscle tissue taken from 129 subjects, when they were aged ~70yr and characterised as disease-free (note the above average longevity in Swedes born circa 1920 compared with US and UK populations). From these samples, 108 yielded RNA of sufficient quality to profile on Affymetrix gene-chips."
Line 8: !Series_overall_design	"Only survival data are used in the paper."
Line 9: !Series_overall_design	"There a

### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the gene expression data
# The gene identifiers appear to be numeric values (e.g., 2315251, 2315373)
# These are likely probe IDs from an Affymetrix microarray rather than standard human gene symbols
# Based on Series_summary mentioning Affymetrix gene-chips and the numeric format,
# these identifiers need to be mapped to human gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 2340731 rows

Gene annotation preview (first few rows):
{'ID': ['2315100', '2315106', '2315109', '2315111', '2315113'], 'GB_LIST': ['NR_024005,NR_034090,NR_024004,AK093685', 'DQ786314', nan, nan, 'DQ786265'], 'SPOT_ID': ['chr1:11884-14409', 'chr1:14760-15198', 'chr1:19408-19712', 'chr1:25142-25532', 'chr1:27563-27813'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['11884', '14760', '19408', '25142', '27563'], 'RANGE_STOP': ['14409', '15198', '19712', '25532', '27813'], 'total_probes': ['20', '8', '4', '4', '4'], 'gene_assignment': ['NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 9 // 15q26.3 // 100288486 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-

### Step 6: Gene Identifier Mapping

In [7]:
# 1. From the gene annotation preview, I can see that:
# - The 'ID' column in gene_annotation matches the probe IDs in gene_data
# - The 'gene_assignment' column contains gene symbol information but needs parsing

print("\nAnalyzing gene assignment data format...")
# Examine the gene_assignment format to understand how to extract gene symbols
gene_assignment_samples = gene_annotation['gene_assignment'].dropna().head(10).tolist()
print("Sample gene assignments:")
for i, sample in enumerate(gene_assignment_samples[:3]):  # Just show first 3 for brevity
    print(f"{i+1}: {sample}")

# 2. Create a mapping between probe IDs and gene symbols
print("\nCreating gene mapping dataframe...")

# First, filter out rows without gene assignments or with placeholder assignments
gene_annotation_filtered = gene_annotation[~gene_annotation['gene_assignment'].isin(['---', 'nan', ''])]
gene_annotation_filtered = gene_annotation_filtered.dropna(subset=['gene_assignment'])

# Create a mapping dataframe with ID and Gene columns
mapping_df = pd.DataFrame()
mapping_df['ID'] = gene_annotation_filtered['ID']
mapping_df['Gene'] = gene_annotation_filtered['gene_assignment']

# Convert the mapping dataframe to the required format
print(f"Initial mapping dataframe shape: {mapping_df.shape}")
print("Sample of mapping dataframe before processing:")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
print("\nApplying gene mapping to convert probe-level expression to gene-level expression...")
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=mapping_df)
print(f"Resulting gene expression dataframe shape: {gene_data.shape}")
print("First few gene symbols in gene expression data:")
print(list(gene_data.index[:10]))

# Save the gene expression data to CSV
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")



Analyzing gene assignment data format...
Sample gene assignments:
1: NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 9 // 15q26.3 // 100288486 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// AK093685 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771
2: ---
3: ---

Creating gene mapping dataframe...


Initial mapping dataframe shape: (33475, 2)
Sample of mapping dataframe before processing:
         ID                                               Gene
0   2315100  NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-As...
10  2315125  NM_001005240 // OR4F17 // olfactory receptor, ...
14  2315147  XM_002343043 // LOC100288692 // protein capicu...
15  2315160  AK303004 // FLJ45445 // hypothetical LOC399844...
16  2315163  AK302511 // LOC100132062 // hypothetical LOC10...

Applying gene mapping to convert probe-level expression to gene-level expression...


Resulting gene expression dataframe shape: (50375, 108)
First few gene symbols in gene expression data:
['A-', 'A-2', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1']


Gene expression data saved to ../../output/preprocess/Longevity/gene_data/GSE48264.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols using NCBI Gene database...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization: {normalized_gene_data.shape[0]} unique gene symbols")
print(f"Sample of normalized gene symbols: {normalized_gene_data.index[:5].tolist()}")

# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load clinical data
print("\nLoading clinical data...")
try:
    clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
    print(f"Loaded clinical data with shape: {clinical_data.shape}")
    is_trait_available = True
except Exception as e:
    print(f"Error loading clinical data: {str(e)}")
    is_trait_available = False

# 3. Link clinical and genetic data
if is_trait_available and is_gene_available:
    print("\nLinking clinical and genetic data...")
    linked_data = geo_link_clinical_genetic_data(clinical_data, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    
    # 4. Handle missing values
    print("\nHandling missing values in linked data...")
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Data shape after handling missing values: {linked_data.shape}")
    
    # 5. Determine if trait and demographic features are severely biased
    print("\nEvaluating trait and demographic feature bias...")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 6. Final validation and saving
    note = "This dataset contains gene expression data from skeletal muscle biopsies of elderly men (age ~70) with longevity as the trait of interest, determined by survival status over a 20-year follow-up period."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )
    
    # Save the linked data if it's usable
    if is_usable:
        print(f"\nSaving linked data to {out_data_file}")
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved successfully!")
    else:
        print(f"\nDataset not usable for {trait} association studies due to bias or quality issues.")
else:
    print("\nCannot create linked data: missing clinical or gene data")
    # Set up dummy linked_data for validation
    linked_data = pd.DataFrame()
    is_biased = True
    
    note = f"Dataset contains {'gene expression data' if is_gene_available else 'no gene data'} but {'lacks' if not is_trait_available else 'has'} clinical information for {trait}."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )
    
    print(f"\nDataset usability for {trait} association studies: {is_usable}")


Normalizing gene symbols using NCBI Gene database...
After normalization: 18910 unique gene symbols
Sample of normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1']


Normalized gene data saved to ../../output/preprocess/Longevity/gene_data/GSE48264.csv

Loading clinical data...
Loaded clinical data with shape: (1, 108)

Linking clinical and genetic data...
Linked data shape: (108, 18911)

Handling missing values in linked data...


Data shape after handling missing values: (76, 18911)

Evaluating trait and demographic feature bias...
For the feature 'Longevity', the least common label is '0.0' with 12 occurrences. This represents 15.79% of the dataset.
The distribution of the feature 'Longevity' in this dataset is fine.


Saving linked data to ../../output/preprocess/Longevity/GSE48264.csv


Linked data saved successfully!
